In [ ]:
#@title
#all imports
import zipfile
# filter warnings
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

from keras.preprocessing import image
from keras.models import Model
from keras.models import model_from_json
from keras.layers import Input

# other imports
from sklearn.preprocessing import LabelEncoder
import numpy as np
import glob
import cv2
import h5py
import os
import json
import datetime
import time

In [ ]:
#connect to drive and unzip
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
#get the files from zipped folders
#here only one folder is unzipped

fid = drive.ListFile({'q':"title='65.zip'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('65.zip')

f.keys()

!unzip 65.zip

Archive:  65.zip
   creating: 65/
  inflating: 65/bcc000000.bmp        
  inflating: 65/bcc000001.bmp        
  inflating: 65/bcc000002.bmp        
  inflating: 65/bcc000003.bmp        
  inflating: 65/bcc000004.bmp        
  inflating: 65/bcc000005.bmp        
  inflating: 65/bcc000006.bmp        
  inflating: 65/bcc000007.bmp        
  inflating: 65/bcc000008.bmp        
  inflating: 65/bcc000009.bmp        
  inflating: 65/bcc000010.bmp        
  inflating: 65/bcc000011.bmp        
  inflating: 65/bcc000012.bmp        
  inflating: 65/bcc000013.bmp        
  inflating: 65/bcc000014.bmp        
  inflating: 65/bcc000015.bmp        
  inflating: 65/bcc000016.bmp        
  inflating: 65/bcc000017.bmp        
  inflating: 65/bcc000018.bmp        
  inflating: 65/bcc000019.bmp        
  inflating: 65/bcc000020.bmp        
  inflating: 65/bcc000021.bmp        
  inflating: 65/bcc000022.bmp        
  inflating: 65/bcc000023.bmp        
  inflating: 65/bcc000024.bmp        
  inflating: 65/

In [ ]:
#data renaming
import os
import cv2
from google.colab.patches import cv2_imshow

cdd= os.getcwd() #returns current working directory
train_path= '/content/65'
train_labels=os.listdir(train_path)

le = LabelEncoder() #Encode target labels with value between 0 and n_classes-1
for tl in train_labels:
  le.fit([tl])

# loop over all the labels in the folder
count = 1
for i, l in enumerate(train_labels):
  #print(i,l)
  save_p = "/content/65"
  count=1
  save_p = save_p + "/"+l
  cur_path = train_path + "/" + l
  k= 0
  #traverse files in a folder
  for image_path in glob.glob(cur_path + "/*.bmp"):
    k= k+1
    sav = save_p + "/" + str(k) +".png"
    #print(sav)
    #print(image_path)
    os.rename(image_path, sav)
    
    #preprocess
    originalImage = cv2.imread(sav)
    #cv2_imshow(originalImage)
    
    grayImage = cv2.cvtColor(originalImage, cv2.COLOR_BGR2GRAY)
    
    grayImage = cv2.GaussianBlur(grayImage,(3,3),2)  
    
    (thresh, blackAndWhiteImage) = cv2.threshold(grayImage, 122, 255, cv2.THRESH_BINARY)
    imagem = cv2.bitwise_not(blackAndWhiteImage)
    
    width = 64
    height = 64
    dim = (width, height)

    # resize image
    resized = cv2.resize(imagem, dim, interpolation = cv2.INTER_AREA) 
    #resized = cv2.resize(blackAndWhiteImage, dim, interpolation = cv2.INTER_AREA) 
    #cv2_imshow(resized)
    cv2.imwrite(sav,resized)

In [ ]:
#@title
#for image_path in glob.glob(cur_path + "/*.jpg"):
              k= k+1
              sav = save_p + "/" + str(k) +".png"
              #print(sav)
              #print(image_path)
              os.rename(image_path, sav)
              
              #preprocess
              originalImage = cv2.imread(sav)
              #cv2_imshow(originalImage)
              
              grayImage = cv2.cvtColor(originalImage, cv2.COLOR_BGR2GRAY)
              
              grayImage = cv2.GaussianBlur(grayImage,(3,3),2)  
              
              (thresh, blackAndWhiteImage) = cv2.threshold(grayImage, 122, 255, cv2.THRESH_BINARY)
              imagem = cv2.bitwise_not(blackAndWhiteImage)
              
              width = 64
              height = 64
              dim = (width, height)

              # resize image
              resized = cv2.resize(imagem, dim, interpolation = cv2.INTER_AREA) 
              #cv2_imshow(resized)
              cv2.imwrite(sav,resized)

In [ ]:
#@title
import os
import cv2
from google.colab.patches import cv2_imshow

save_p = "/content/65"

cur_path = "/content/65"
k= 0
  #traverse files in a folder
for image_path in glob.glob(cur_path + "/*.png"):
  k= k+1
  sav = save_p + "/" + str(k) +".png"
    #print(sav)
    #print(image_path)
  os.rename(image_path, sav)
    
    #preprocess
  originalImage = cv2.imread(sav)
    #cv2_imshow(originalImage)
    
  grayImage = cv2.cvtColor(originalImage, cv2.COLOR_BGR2GRAY)
    
  grayImage = cv2.GaussianBlur(grayImage,(3,3),2)  
    
  (thresh, blackAndWhiteImage) = cv2.threshold(grayImage, 122, 255, cv2.THRESH_BINARY)
  imagem = cv2.bitwise_not(blackAndWhiteImage)
    
  width = 64
  height = 64
  dim = (width, height)

    # resize image
  resized = cv2.resize(imagem, dim, interpolation = cv2.INTER_AREA) 
    #cv2_imshow(resized)
  cv2.imwrite(sav,resized)

In [ ]:
#@title
import numpy as np
import os
import cv2
import random
from google.colab.patches import cv2_imshow
from skimage import io 
from skimage.transform import rotate, AffineTransform, warp

#functions
def anticlockwise_rotation(image):
    angle= random.randint(0,30) #change to 30 later
    return rotate(image, angle)

def clockwise_rotation(image):
    angle= random.randint(0,30) #change to 30 later
    return rotate(image, -angle)

def rotate(img, angle):   
    ag= 0
    angle = int(random.uniform(ag, angle))   
    h, w = img.shape[:2]   
    M = cv2.getRotationMatrix2D((int(w/2), int(h/2)), angle, 1)   
    img = cv2.warpAffine(img, M, (w, h))   
    return img
      
def ltransform(img):
    rows,cols = img.shape[:2]
    M = np.float32([[1,0,-5],[0,1,-2]])
    img = cv2.warpAffine(img,M,(cols,rows))
    return img

def rtransform(img):
    rows,cols = img.shape[:2]
    M = np.float32([[1,0,5],[0,1,2]])
    img = cv2.warpAffine(img,M,(cols,rows))
    return img

def augment(path, x, t):
    #augment
    self=cv2.imread(path)
    im_crot= clockwise_rotation(self)
    im_arot= anticlockwise_rotation(self)
    im_ltran = ltransform(self)
    im_rtran = rtransform(self)
    
    #cv2_imshow(im_crot)
    #cv2_imshow(im_arot)
    #cv2_imshow(im_ltran)
    #cv2_imshow(im_rtran)
    
    #writeImage
    save_p='/content/65/'+ str(t)+'/'
    cv2.imwrite(save_p+str(x)+'_crot.png',im_crot)
    cv2.imwrite(save_p+str(x)+'_arot.png',im_arot)
    cv2.imwrite(save_p+str(x)+'_ltran.png',im_ltran)
    cv2.imwrite(save_p+str(x)+'_rtran.png',im_rtran)
    cv2.waitKey(0)

#main
t=0
while t<243:
  train_path = '/content/65/'+str(t)
  train_labels = os.listdir(train_path) #set labels to names of the entries in the directory

  # encode the labels
  le = LabelEncoder() #Encode target labels with value between 0 and n_classes-1
  le.fit([tl for tl in train_labels]) #Fit label encoder

  # loop over all the labels in the folder
  for i, l in enumerate(train_labels):
    #print (i,l)
    
    if ".png" not in l:
      continue
    
    
    cur_path = train_path + '/' + l
    ori = cv2.imread(cur_path)
    #cv2_imshow(ori)
    x = int(l.strip(".png"))
    augment(cur_path, x, t)
  
  t=t+1

In [ ]:
#zip the folder
!zip -r /content/f65.zip /content/65